In [1]:
!export PYTHONPATH=.:$PYTHONPATH

In [45]:
from odi.feature_engg import feature_extractor as fe
from odi.feature_engg import util as cricutil
from odi.preprocessing import rank
from odi.retrain import create_train_test_ci as ctt
from odi.model_util import odi_util as outil
import pandas as pd
from datetime import datetime
import math
import numpy as np
from tqdm.notebook import tqdm
import os

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline


from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,accuracy_score

In [46]:
import pickle

In [47]:
train_x=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.second_innings_base_train_x),'rb'))
train_y=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.second_innings_base_train_y),'rb'))

test_x=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.second_innings_base_test_x),'rb'))
test_y=pickle.load(open(os.path.join(ctt.TRAIN_TEST_DIR,ctt.second_innings_base_test_y),'rb'))

cols = pickle.load(open(os.path.join(outil.DEV_DIR, ctt.second_innings_base_columns), 'rb'))

In [48]:
train_x.shape

(441, 57)

In [49]:
test_x.shape

(75, 57)

In [50]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [51]:
# pipe = Pipeline([('scaler', StandardScaler()), ('regression', LinearRegression())])
# sfs = SequentialFeatureSelector(pipe, n_features_to_select=10)
# sfs.fit(train_x, train_y)

In [52]:
train_df = pd.DataFrame(train_x)
train_df.columns= cols
train_df['win']=train_y

test_df = pd.DataFrame(test_x)
test_df.columns = cols
test_df['win']=test_y

In [53]:
train_df.isnull().sum()

team_score                                       0
opponent_score                                   0
opponent_base                                    0
opponent_trend                                   0
opponent_trend_predict                           0
opponent_mean                                    0
location_base                                    0
location_trend                                   0
location_trend_predict                           0
location_mean                                    0
current_base                                     0
current_trend                                    0
current_trend_predict                            0
current_mean                                     0
run_factor                                       0
score_sum_7                                      0
score_sum_6                                      0
score_sum_5                                      0
score_sum_4                                      0
score_sum_3                    

In [54]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [55]:
#np.array(train_df)

In [56]:
pipe = Pipeline([('scaler', StandardScaler()), ('regression', LinearRegression())])
sfs = SequentialFeatureSelector(pipe, n_features_to_select=10)
sfs.fit(train_df.drop(columns='win'), train_df['win'])

SequentialFeatureSelector(estimator=Pipeline(steps=[('scaler',
                                                     StandardScaler()),
                                                    ('regression',
                                                     LinearRegression())]),
                          n_features_to_select=10)

In [57]:
print("=======overall improtance=========")
selected_cols = []
for idx in np.where(sfs.get_support())[0]:
    print(cols[idx])
    selected_cols.append(cols[idx])

=======overall improtance=========
team_score
current_trend_predict
score_sum_weighted_by_contribution
score_sum_weighted_by_effectiveness
score_mean_weighted_by_correlation_4
bowler_score_sum
bowler_score_mean
bowler_score_sum_weighted_by_wr_contribution
bowler_score_mean_weighted_by_wr_contribution
target_score


In [58]:
train_x_selected = np.array(train_df[selected_cols])
test_x_selected = np.array(test_df[selected_cols])

train_y_selected = np.array(train_df['win'])
test_y_selected = np.array(test_df['win'])

In [69]:
train_pipe = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('regression', LogisticRegression())])
#train_pipe = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('svc',SVR(C=0.5, epsilon=0.2))])



In [70]:
train_pipe.fit(train_x_selected,train_y_selected)

Pipeline(steps=[('scaler', StandardScaler()),
                ('polynom', PolynomialFeatures(degree=1)),
                ('regression', LogisticRegression())])

In [71]:
train_predict = train_pipe.predict(train_x_selected)
test_predict = train_pipe.predict(test_x_selected)

In [72]:
accuracy_score(train_y_selected,train_predict)



0.7663551401869159

In [73]:
accuracy_score(test_y_selected,test_predict)

0.72

In [74]:
# SS_Residual = sum((train_y_selected-train_predict)**2)    
# SS_Total = sum((train_y_selected-np.mean(train_y_selected))**2) 
# r_squared = 1 - (float(SS_Residual))/SS_Total
# adjusted_r_squared = 1 - (1-r_squared)*(len(train_y_selected)-1)/(len(train_y_selected)-train_x_selected.shape[1]-1)

# r_squared,adjusted_r_squared



In [75]:
train_x_selected_all = np.array(train_df[cols])
test_x_selected_all = np.array(test_df[cols])

train_y_selected = np.array(train_df['win'])
test_y_selected = np.array(test_df['win'])

In [76]:
train_pipe_all = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('regression', LogisticRegression())])

In [77]:
train_pipe_all.fit(train_x_selected_all,train_y_selected)

Pipeline(steps=[('scaler', StandardScaler()),
                ('polynom', PolynomialFeatures(degree=1)),
                ('regression', LogisticRegression())])

In [78]:
train_predict_all = train_pipe_all.predict(train_x_selected_all)
test_predict_all = train_pipe_all.predict(test_x_selected_all)

In [79]:
accuracy_score(train_y_selected,train_predict_all)

0.7710280373831776

In [80]:
accuracy_score(test_y_selected,test_predict_all)

0.6933333333333334

In [215]:
cols

['team_score',
 'opponent_score',
 'opponent_base',
 'opponent_trend',
 'opponent_trend_predict',
 'opponent_mean',
 'location_base',
 'location_trend',
 'location_trend_predict',
 'location_mean',
 'current_base',
 'current_trend',
 'current_trend_predict',
 'current_mean',
 'run_factor',
 'score_sum_7',
 'score_sum_6',
 'score_sum_5',
 'score_sum_4',
 'score_sum_3',
 'score_max',
 'score_sum_weighted_by_correlation',
 'score_sum_weighted_by_contribution',
 'score_sum_weighted_by_effectiveness',
 'score_sum_weighted_by_log',
 'score_mean_weighted_by_correlation',
 'score_mean_weighted_by_contribution',
 'score_mean_weighted_by_effectiveness',
 'score_mean_weighted_by_log',
 'score_sum_weighted_by_correlation_4',
 'score_sum_weighted_by_contribution_4',
 'score_sum_weighted_by_effectiveness_4',
 'score_sum_weighted_by_log_4',
 'score_mean_weighted_by_correlation_4',
 'score_mean_weighted_by_contribution_4',
 'score_mean_weighted_by_effectiveness_4',
 'score_mean_weighted_by_log_4',
 's

In [223]:
# handpicked_cols = ['score_sum_weighted_by_correlation','score_sum_weighted_by_contribution',
#                    'team_score','bowler_score_sum',
#                   'location_base','current_trend_predict','current_mean']

handpicked_cols = ['team_score','opponent_score','score_sum_weighted_by_correlation','score_sum_weighted_by_contribution',
                   'score_sum_weighted_by_effectiveness','bowler_score_sum_weighted_by_contribution','bowler_score_sum_weighted_by_wr_contribution',
                   'team_score','bowler_score_sum',
                  'location_base','current_trend_predict','current_mean']



In [224]:
train_x_selected_hp = np.array(train_df[handpicked_cols])
test_x_selected_hp = np.array(test_df[handpicked_cols])

train_y_selected = np.array(train_df['runs'])
test_y_selected = np.array(test_df['runs'])

In [225]:
train_pipe_hp = Pipeline([('scaler', StandardScaler()), ('polynom',PolynomialFeatures(1)),('regression', LinearRegression())])

In [226]:
train_pipe_hp.fit(train_x_selected_hp,train_y_selected)

Pipeline(steps=[('scaler', StandardScaler()),
                ('polynom', PolynomialFeatures(degree=1)),
                ('regression', LinearRegression())])

In [227]:
train_predict_hp = train_pipe_hp.predict(train_x_selected_hp)
test_predict_hp = train_pipe_hp.predict(test_x_selected_hp)

In [228]:
mean_absolute_error(train_y_selected,train_predict_hp), mean_absolute_percentage_error(train_y_selected,train_predict_hp)



(44.106140827044705, 0.19740297483154506)

In [229]:
mean_absolute_error(test_y_selected,test_predict_hp), mean_absolute_percentage_error(test_y_selected,test_predict_hp)



(53.42038480330737, 0.22505873059786413)

In [81]:
tq1 = np.quantile(train_df['team_score'],0.25)
tq2 = np.quantile(train_df['team_score'],0.50)
tq3 = np.quantile(train_df['team_score'],0.75)

In [82]:
train_df_q1 = train_df[train_df['team_score']<tq1]
train_df_q2 = train_df[train_df['team_score']<tq2]
train_df_q3 = train_df[train_df['team_score']<tq3]
train_df_q4 = train_df[train_df['team_score']>=tq3]

test_df_q1 = test_df[test_df['team_score']<tq1]
test_df_q2 = test_df[test_df['team_score']<tq2]
test_df_q3 = test_df[test_df['team_score']<tq3]
test_df_q4 = test_df[test_df['team_score']>=tq3]

In [86]:
train_pipe_q1 = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('cls', LogisticRegression())])
train_pipe_q2 = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('slc', LogisticRegression())])
train_pipe_q3 = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('cls', LogisticRegression())])
train_pipe_q4 = Pipeline([('scaler', StandardScaler()),('polynom',PolynomialFeatures(1)), ('cls', LogisticRegression())])



In [85]:
train_x_q1 = np.array(train_df_q1[cols])
test_x_q1 = np.array(test_df_q1[cols])
train_y_q1 = np.array(train_df_q1['win'])
test_y_q1 = np.array(test_df_q1['win'])

train_x_q2 = np.array(train_df_q2[cols])
test_x_q2 = np.array(test_df_q2[cols])
train_y_q2 = np.array(train_df_q2['win'])
test_y_q2 = np.array(test_df_q2['win'])

train_x_q3 = np.array(train_df_q3[cols])
test_x_q3 = np.array(test_df_q3[cols])
train_y_q3 = np.array(train_df_q3['win'])
test_y_q3 = np.array(test_df_q3['win'])

train_x_q4 = np.array(train_df_q4[cols])
test_x_q4 = np.array(train_df_q4[cols])
train_y_q4 = np.array(train_df_q4['win'])
test_y_q4 = np.array(train_df_q4['win'])

In [87]:
train_pipe_q1.fit(train_x_q1,train_y_q1)
train_pipe_q2.fit(train_x_q2,train_y_q2)
train_pipe_q3.fit(train_x_q3,train_y_q3)
train_pipe_q4.fit(train_x_q4,train_y_q4)

Pipeline(steps=[('scaler', StandardScaler()),
                ('polynom', PolynomialFeatures(degree=1)),
                ('cls', LogisticRegression())])

In [89]:
train_predict_q1 = train_pipe_q1.predict(train_x_q1)
train_predict_q2 = train_pipe_q2.predict(train_x_q2)
train_predict_q3 = train_pipe_q3.predict(train_x_q3)
train_predict_q4 = train_pipe_q4.predict(train_x_q4)

test_predict_q1 = train_pipe_q1.predict(test_x_q1)
test_predict_q2 = train_pipe_q2.predict(test_x_q2)
test_predict_q3 = train_pipe_q3.predict(test_x_q3)
test_predict_q4 = train_pipe_q4.predict(test_x_q4)

In [90]:
accuracy_score(train_y_q1,train_predict_q1), accuracy_score(test_y_q1,test_predict_q1)

(0.9245283018867925, 0.9245283018867925)

In [91]:
accuracy_score(train_y_q2,train_predict_q2), accuracy_score(test_y_q2,test_predict_q2)

(0.8364485981308412, 0.8364485981308412)

In [92]:
accuracy_score(train_y_q3,train_predict_q3), accuracy_score(test_y_q3,test_predict_q3)

(0.7881619937694704, 0.7881619937694704)

In [93]:
accuracy_score(train_y_q4,train_predict_q4), accuracy_score(test_y_q4,test_predict_q4)

(0.8691588785046729, 0.8691588785046729)